In [ ]:
#!pwsh
ls

In [1]:
#!pwsh
#!time
Remove-Item /Users/mscottford/src/corgibytes/freshli/Freshli/bin/Debug/*.nupkg
if (Test-Path /Users/mscottford/.nuget/packages/freshli) {
  Remove-Item /Users/mscottford/.nuget/packages/freshli/* -Recurse -Force
}
dotnet build
dotnet pack
Get-ChildItem -Recurse *.nupkg

The below script needs to be able to find the current output cell; this is an easy method to get it.

Microsoft (R) Build Engine version 16.6.0+5ff7b0c9e for .NET Core
Copyright (C) Microsoft Corporation. All rights reserved.

  Determining projects to restore...
  Restored /Users/mscottford/src/corgibytes/freshli/Freshli/Freshli.csproj (in 251 ms).
  Restored /Users/mscottford/src/corgibytes/freshli/Freshli.Test/Freshli.Test.csproj (in 318 ms).
VersionInfo.cs(200,32): warning CS8632: The annotation for nullable reference types should only be used in code within a '#nullable' annotations context. [/Users/mscottford/src/corgibytes/freshli/Freshli/Freshli.csproj]
  Freshli -> /Users/mscottford/src/corgibytes/freshli/Freshli/bin/Debug/netcoreapp3.1/Freshli.dll
  Freshli.Test -> /Users/mscottford/src/corgibytes/freshli/Freshli.Test/bin/Debug/netcoreapp3.1/Freshli.Test.dll
  Archive:  nokotest.zip
    inflating: nokotest/Gemfile        
    inflating: nokotest/Gemfile.lock   
    inflating: nokotest/.git/config    
   extracting: nokotest/.git/objects/0d/8f4f864a22eac5f72153cf1d77fc9791796e

Wall time: 5544.7052ms

Wall time: 34.9161ms

In [3]:
#i nuget:/Users/mscottford/src/corgibytes/freshli/Freshli/bin/Debug
#r nuget:freshli,1.0.0-dev

Restore sources /Users/mscottford/src/corgibytes/freshli/Freshli/bin/Debug

Installed package freshli version 1.0.0-dev-20200626t0950

In [4]:
using XPlot.Plotly;

using Freshli;
using Freshli.Languages.Ruby;
using Freshli.Languages.Php;
using Freshli.Languages.Python;

ManifestFinder.Register<RubyBundlerManifestFinder>();
ManifestFinder.Register<PhpComposerManifestFinder>();
ManifestFinder.Register<PipRequirementsTxtManifestFinder>();

FileHistoryFinder.Register<GitFileHistoryFinder>();

In [5]:
PlotlyChart CreateLineGraphFor(Dictionary<string, IList<MetricsResult>> projects) {
    var lineSeries = projects.Select(p => new Scattergl { 
        name = p.Key, 
        x = p.Value.Select(r => r.Date),
        y = p.Value.Select(r => r.LibYear.Total)
    });


    var chart = Chart.Plot(lineSeries.ToArray());
    chart.WithTitle("LibYear over time");
    return chart;
}

In [6]:
PlotlyChart CreateStackedAreaFor(IList<MetricsResult> project) {
    var names = project.Select(r => r.LibYear.Select(p => p.Name).ToArray()).Aggregate(
        new HashSet<string>(), 
        (hash, values) => {
            foreach(var value in values) {
                hash.Add(value);
            }
            return hash;
        }
    );
    
    var areaSeries = new List<Scatter>();

    foreach (var name  in names) {
        var projectData = project.Where(r => r.LibYear.Any(p => p.Name == name));
        var projectDates = projectData.Select(r => r.Date);
        var projectLibYearValues = projectData.Select(r => r.LibYear[name].Value);

        areaSeries.Add(new Scatter {
            name = name,
            x = projectDates,
            y = projectLibYearValues,
            fill = "tonexty",
            stackgroup = "one"
        });
    }

    var chart = Chart.Plot(areaSeries.ToArray());
    chart.WithTitle("LibYear over time per dependency");
    return chart;
}

In [7]:
var runner = new Runner();

In [8]:
var projects = new Dictionary<string, IList<MetricsResult>>();

In [9]:
#!time
projects["ansible"] = runner.Run("https://github.com/ansible/ansible");
projects["Detectron"] = runner.Run("https://github.com/facebookresearch/Detectron");                                   
projects["face_recognition"] = runner.Run("https://github.com/ageitgey/face_recognition");
projects["glances"] = runner.Run("https://github.com/nicolargo/glances");
projects["http-prompt"] = runner.Run("https://github.com/eliangcs/http-prompt");
projects["pyspider"] = runner.Run("https://github.com/binux/pyspider");
projects["Real-Time-Voice-Cloning"] = runner.Run("https://github.com/CorentinJ/Real-Time-Voice-Cloning");
projects["snallygaster"] = runner.Run("https://github.com/hannob/snallygaster");
projects["statsmodels"] = runner.Run("https://github.com/statsmodels/statsmodels");

Wall time: 359063.7866ms

In [11]:
projects["pyspider"] = runner.Run("https://github.com/binux/pyspider");
display(CreateLineGraphFor(projects))

In [12]:
display(CreateStackedAreaFor(projects["pyspider"]))

In [13]:
var projects = new Dictionary<string, IList<MetricsResult>>();
projects["feedbin"] = runner.Run("https://github.com/feedbin/feedbin");

display(CreateLineGraphFor(projects))

In [14]:
display(CreateStackedAreaFor(projects["feedbin"]))

In [ ]:
#!time
projects["feedbin"] = runner.Run("https://github.com/feedbin/feedbin");
projects["lobsters"] = runner.Run("https://github.com/lobsters/lobsters");                                   
projects["Portus"] = runner.Run("https://github.com/SUSE/Portus");
projects["octobox"] = runner.Run("https://github.com/octobox/octobox");
projects["typhoon"] = runner.Run("https://github.com/culanth/typhoon");
projects["sprinkle"] = runner.Run("https://github.com/sprinkle-tool/sprinkle");

Application is shutting down...


In [ ]:
display(CreateLineGraphFor(projects))

In [ ]:
display(CreateStackedAreaFor(projects["typhoon"]))

In [3]:
#!pwsh
dotnet nuget delete freshli 1.0.0-dev

error: Source parameter was not specified.


Usage: dotnet nuget delete [arguments] [options]

Arguments:
  [root]  The Package Id and version.

Options:
  -h|--help               Show help information
  --force-english-output  Forces the application to run using an invariant, English-based culture.
  -s|--source <source>    Package source (URL, UNC/folder path or package source name) to use. Defaults to DefaultPushSource if specified in NuGet.Config.
  --non-interactive       Do not prompt for user input or confirmations.
  -k|--api-key <apiKey>   The API key for the server.
  --no-service-endpoint   Does not append "api/v2/package" to the source URL.
  --interactive           Allow the command to block and require manual action for operations like authentication.
